In [ ]:
#O*NET-SOC

In [ ]:
!pip install tensorflow numpy -q

: 

In [3]:
#!pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [2]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'distutils'

In [3]:
import tensorflow as tf
import numpy as np

class SparseEvolutionaryModel(tf.keras.Model):
    def __init__(self, input_dim, output_dim, sparsity=0.1):
        super(SparseEvolutionaryModel, self).__init__()
        self.sparsity = sparsity
        self.dense = tf.keras.layers.Dense(output_dim, use_bias=False)
        # Manually initialize weights to zero to simulate sparsity
        self.dense.build((None, input_dim))  # Initialize weights
        weights = np.zeros(self.dense.kernel.shape)
        self.dense.kernel.assign(weights)

    def call(self, inputs):
        return self.dense(inputs)

    def prune_and_grow_weights(self):
        # Convert weights to numpy for manipulation
        weights = self.dense.kernel.numpy()
        nonzero_indices = weights.nonzero()
        num_nonzero = len(nonzero_indices[0])
        num_to_prune = int(num_nonzero * self.sparsity)

        # Prune
        if num_nonzero > 0:
            abs_weights = np.abs(weights)
            flat_indices = np.argpartition(abs_weights.flatten(), num_to_prune)[:num_to_prune]
            weights.flat[flat_indices] = 0

        # Grow
        zero_indices = np.where(weights == 0)
        num_possible_grows = len(zero_indices[0])
        if num_possible_grows > 0:
            indices_to_grow = np.random.choice(np.arange(num_possible_grows), size=num_to_prune, replace=False)
            weights[zero_indices[0][indices_to_grow], zero_indices[1][indices_to_grow]] = np.random.randn(num_to_prune) * 0.1

        # Assign modified weights back to the layer
        self.dense.kernel.assign(weights)

# Dummy data
input_dim = 10
output_dim = 1
X = np.random.randn(1000, input_dim).astype(np.float32)
y = np.random.randn(1000, output_dim).astype(np.float32)

# Model and optimizer
model = SparseEvolutionaryModel(input_dim, output_dim, sparsity=0.1)
optimizer = tf.keras.optimizers.Adam()

# Training loop
for epoch in range(10):
    with tf.GradientTape() as tape:
        predictions = model(X)
        loss = tf.reduce_mean((predictions - y) ** 2)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # After each epoch, prune and grow weights
    model.prune_and_grow_weights()
    
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")


: 